In [19]:
from __future__ import print_function
import csv
import pandas as pd
import numpy as np
import tensorflow as tf

In [20]:
# Load data
train = pd.read_csv('../data/datosEntrenamiento.csv', header = None)
test = pd.read_csv('../data/datosPrueba.csv', header = None)
validation = pd.read_csv('../data/datosValidacion.csv', header = None)

In [21]:
train=train.rename(columns = {36:'class'})
test=test.rename(columns = {36:'class'})

In [22]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,class
0,76,94,98,76,72,85,90,72,68,85,...,62,71,79,85,62,67,75,85,62,2
1,68,89,86,72,68,85,90,76,68,94,...,67,71,75,81,62,67,75,85,71,2
2,92,115,115,94,92,111,120,91,84,106,...,92,93,107,109,92,93,111,113,92,1
3,84,106,111,87,84,98,111,87,84,98,...,92,93,107,113,87,93,107,113,87,1
4,88,111,115,91,88,111,120,87,88,111,...,87,88,107,109,87,88,107,109,87,1


In [23]:
train_class_dummies = pd.get_dummies(list(train['class']))
train_class_dummies.columns = ['class1','class2','class3','class4','class5','class6']
test_class_dummies = pd.get_dummies(list(test['class']))
test_class_dummies.columns = ['class1','class2','class3','class4','class5','class6']

In [24]:
train = pd.concat([train,train_class_dummies], axis = 1)
test = pd.concat([test,test_class_dummies], axis = 1)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,class,class1,class2,class3,class4,class5,class6
0,92,115,120,94,84,102,106,79,84,102,...,107,113,87,1,1,0,0,0,0,0
1,84,102,102,83,80,102,102,79,84,94,...,99,104,79,1,1,0,0,0,0,0
2,84,94,102,79,80,94,98,76,80,102,...,107,109,87,1,1,0,0,0,0,0
3,80,94,98,76,80,102,102,79,76,102,...,107,109,87,1,1,0,0,0,0,0
4,76,102,106,83,76,102,106,87,80,98,...,103,104,79,1,1,0,0,0,0,0


In [25]:
x_test = test.drop(test.columns[list(range(36,43))], axis = 1).values
y_test = test.drop(test.columns[list(range(37))],axis = 1).values
x_train = train.drop(train.columns[list(range(36,43))], axis = 1).values
y_train = train.drop(test.columns[list(range(37))],axis = 1).values

In [26]:
y_test.shape

(1287, 6)

In [27]:
# Parameters
learning_rate = 0.0004909617858915018
num_steps = 869
batch_size = 13
display_step = num_steps / 10

In [28]:
# Network Parameters
n_hidden_1 = 342
n_hidden_2 = 342
n_hidden_3 = 342
num_input = 36
num_classes = 6

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])

In [29]:
Y.shape

TensorShape([Dimension(None), Dimension(6)])

In [30]:
X.shape

TensorShape([Dimension(None), Dimension(36)])

In [13]:
y_train.shape

(4504, 6)

In [14]:
x_train.shape

(4504, 36)

In [15]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [16]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_3, weights['out']) + biases['out']
    return out_layer

In [17]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [18]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x = x_train
        batch_y = y_train
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test,
                                      Y: y_test}))

Step 1, Minibatch Loss= 4259654.5000, Training Accuracy= 0.108
Step 86, Minibatch Loss= 111071.3438, Training Accuracy= 0.579
Step 172, Minibatch Loss= 55646.3164, Training Accuracy= 0.659
Step 258, Minibatch Loss= 38002.8438, Training Accuracy= 0.693
Step 344, Minibatch Loss= 31212.6758, Training Accuracy= 0.707
Step 430, Minibatch Loss= 26573.1270, Training Accuracy= 0.719
Step 516, Minibatch Loss= 22762.0742, Training Accuracy= 0.727
Step 602, Minibatch Loss= 133101.3281, Training Accuracy= 0.462
Step 688, Minibatch Loss= 22935.7695, Training Accuracy= 0.754
Step 774, Minibatch Loss= 16602.7812, Training Accuracy= 0.751
Step 860, Minibatch Loss= 17892.4258, Training Accuracy= 0.691
Optimization Finished!
Testing Accuracy: 0.70240873
